In [301]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import re

train: 기간 2016-02-01~2021-01-26   
test: 기간 2021-01-27~2021-04-08

In [302]:
train = pd.read_csv('D:/김동영/11_Github/mygit-1/데이터사이언스수업/과제/최종과제/235743_구내식당 식사 인원 예측 AI 경진대회_data/train.csv')
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [303]:
train['중식메뉴'] = train['중식메뉴'].str.replace('삽겹', '삼겹')

In [304]:
test = pd.read_csv('D:/김동영/11_Github/mygit-1/데이터사이언스수업/과제/최종과제/235743_구내식당 식사 인원 예측 AI 경진대회_data/test.csv')
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [305]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['요일'] = train['일자'].dt.day_name().str[:2].map({'Mo' : 5, 'Tu' : 4, 'We' : 3, 'Th' : 2, 'Fr' : 1})
test['요일'] = test['일자'].dt.day_name().str[:2].map({'Mo' : 5, 'Tu' : 4, 'We' : 3, 'Th' : 2, 'Fr' : 1})

train['month'] = train['일자'].dt.month
test['month'] = test['일자'].dt.month

# 코로나 관련 변수 생성, 날짜를 확인해본 결과 코로나가 퍼지던 시기인 2020년 2월 23일에 현본사소속재택근무자수가 생겨났다. 
train['corona'] = [1 if x > 0 else 0 for x in train['현본사소속재택근무자수']]
test['corona'] = [1 if x > 0 else 0 for x in test['현본사소속재택근무자수']]

In [306]:
# 앞에 
train['shift_1day'] = train.일자.shift(-1)
test['shift_1day'] = test.일자.shift(-1)

train.at[1204, 'shift_1day'] = datetime(2021,1,27)
test.at[49, 'shift_1day'] = datetime(2021,4,12)

train['day_gap'] = (train.shift_1day - train.일자).astype(str)
test['day_gap'] = (test.shift_1day - test.일자).astype(str)

train.at[0, 'day_gap'] = '1 days'
test.at[0, 'day_gap'] = '1 days'

In [307]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,month,corona,shift_1day,day_gap
0,2016-02-01,5,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,2,0,2016-02-02,1 days
1,2016-02-02,4,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,2,0,2016-02-03,1 days
2,2016-02-03,3,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,2,0,2016-02-04,1 days
3,2016-02-04,2,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,2,0,2016-02-05,1 days
4,2016-02-05,1,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,2,0,2016-02-11,6 days


In [308]:
# 휴가 및 공휴일 여부를 판단하는 함수, shift_1day와 일자 사이의 차이를 계산하여 휴가 및 공휴일 여부를 판단
# 휴가 및 공휴일이 존재했다면 1, 그렇지 않다면 0을 반환
def get_holiday_score(x) :
    
    s = int(re.sub(r'[^0-9]', '', x))
    if s == 1 :
        return 0
    else :
        return 1

In [309]:
train['day_gap'] = train.day_gap.apply(get_holiday_score)
test['day_gap'] = test.day_gap.apply(get_holiday_score)

In [310]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,month,corona,shift_1day,day_gap
0,2016-02-01,5,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,2,0,2016-02-02,0
1,2016-02-02,4,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,2,0,2016-02-03,0
2,2016-02-03,3,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,2,0,2016-02-04,0
3,2016-02-04,2,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,2,0,2016-02-05,0
4,2016-02-05,1,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,2,0,2016-02-11,1


### 음식 메뉴 전처리

In [311]:
train['중식메뉴'] = train['중식메뉴'].str.split(' ')
train['석식메뉴'] = train['석식메뉴'].str.split(' ')

test['중식메뉴'] = test['중식메뉴'].str.split(' ')
test['석식메뉴'] = test['석식메뉴'].str.split(' ')

In [312]:
train['중식메뉴'] = train['중식메뉴'].apply(lambda x: re.sub(r',\s*,', ',', x) if isinstance(x, str) else x)
train['석식메뉴'] = train['석식메뉴'].apply(lambda x: re.sub(r',\s*,', ',', x) if isinstance(x, str) else x)

In [313]:
# train.head()

def remove_parentheses_items(menu_list):
  return [item for item in menu_list if '(' not in item and ')' not in item]

train['중식메뉴'] = train['중식메뉴'].apply(lambda x: [item for item in x.split() if '(' not in item and ')' not in item] if isinstance(x, str) else x)
train['석식메뉴'] = train['석식메뉴'].apply(lambda x: [item for item in x if '(' not in item and ')' not in item] if isinstance(x, list) else x)

test['중식메뉴'] = test['중식메뉴'].apply(lambda x: [item for item in x.split() if '(' not in item and ')' not in item] if isinstance(x, str) else x)
test['석식메뉴'] = test['석식메뉴'].apply(lambda x: [item for item in x.split() if '(' not in item and ')' not in item] if isinstance(x, str) else x)

# 괄호가 포함된 단어 제거 함수
def remove_parentheses_items(menu_list):
  if isinstance(menu_list, list):
    return [item for item in menu_list if '(' not in item and ')' not in item]
  return menu_list

train['중식메뉴'] = train['중식메뉴'].apply(remove_parentheses_items)
train['석식메뉴'] = train['석식메뉴'].apply(remove_parentheses_items)
test['중식메뉴'] = test['중식메뉴'].apply(remove_parentheses_items)
test['석식메뉴'] = test['석식메뉴'].apply(remove_parentheses_items)

In [314]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,month,corona,shift_1day,day_gap
0,2016-02-01,5,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"[쌀밥/잡곡밥, 오징어찌개, , 쇠불고기, 계란찜, , 청포묵무침, , 요구르트, ...","[쌀밥/잡곡밥, 육개장, , 자반고등어구이, , 두부조림, , 건파래무침, , 포기...",1039.0,331.0,2,0,2016-02-02,0
1,2016-02-02,4,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"[쌀밥/잡곡밥, 김치찌개, , 가자미튀김, , 모둠소세지구이, , 마늘쫑무침, , ...","[콩나물밥*양념장, 어묵국, , 유산슬, 아삭고추무침, , 바나나, , 포기김치, ]",867.0,560.0,2,0,2016-02-03,0
2,2016-02-03,3,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"[카레덮밥, 팽이장국, , 치킨핑거, 쫄면야채무침, , 견과류조림, , 요구르트, ...","[쌀밥/잡곡밥, 청국장찌개, , 황태양념구이, 고기전, 새송이버섯볶음, , 포기김치, ]",1017.0,573.0,2,0,2016-02-04,0
3,2016-02-04,2,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","[쌀밥/잡곡밥, 쇠고기무국, , 주꾸미볶음, , 부추전, , 시금치나물, , 요구르...","[미니김밥*겨자장, 우동, , 멕시칸샐러드, , 군고구마, , 무피클, , 포기김치, ]",978.0,525.0,2,0,2016-02-05,0
4,2016-02-05,1,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"[쌀밥/잡곡밥, 떡국, , 돈육씨앗강정, 우엉잡채, , 청경채무침, , 요구르트, ...","[쌀밥/잡곡밥, 차돌박이찌개, 닭갈비, 감자소세지볶음, , 콩나물무침, , 포기김치, ]",925.0,330.0,2,0,2016-02-11,1


In [315]:
def clean_menu_list(menu_list):
    if isinstance(menu_list, list):
        return [item for item in menu_list if item and item != ',' and item.strip(',') != '']
    return menu_list

train['중식메뉴'] = train['중식메뉴'].apply(clean_menu_list)
train['석식메뉴'] = train['석식메뉴'].apply(clean_menu_list)

In [316]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,month,corona,shift_1day,day_gap
0,2016-02-01,5,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"[쌀밥/잡곡밥, 오징어찌개, 쇠불고기, 계란찜, 청포묵무침, 요구르트, 포기김치]","[쌀밥/잡곡밥, 육개장, 자반고등어구이, 두부조림, 건파래무침, 포기김치]",1039.0,331.0,2,0,2016-02-02,0
1,2016-02-02,4,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"[쌀밥/잡곡밥, 김치찌개, 가자미튀김, 모둠소세지구이, 마늘쫑무침, 요구르트, 배추...","[콩나물밥*양념장, 어묵국, 유산슬, 아삭고추무침, 바나나, 포기김치]",867.0,560.0,2,0,2016-02-03,0
2,2016-02-03,3,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"[카레덮밥, 팽이장국, 치킨핑거, 쫄면야채무침, 견과류조림, 요구르트, 포기김치]","[쌀밥/잡곡밥, 청국장찌개, 황태양념구이, 고기전, 새송이버섯볶음, 포기김치]",1017.0,573.0,2,0,2016-02-04,0
3,2016-02-04,2,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","[쌀밥/잡곡밥, 쇠고기무국, 주꾸미볶음, 부추전, 시금치나물, 요구르트, 포기김치]","[미니김밥*겨자장, 우동, 멕시칸샐러드, 군고구마, 무피클, 포기김치]",978.0,525.0,2,0,2016-02-05,0
4,2016-02-05,1,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"[쌀밥/잡곡밥, 떡국, 돈육씨앗강정, 우엉잡채, 청경채무침, 요구르트, 포기김치]","[쌀밥/잡곡밥, 차돌박이찌개, 닭갈비, 감자소세지볶음, 콩나물무침, 포기김치]",925.0,330.0,2,0,2016-02-11,1


확인해본 결과 중식, 석식에서 메인 메뉴는 2번째(국), 3번째(단백질), 4번째 반찬이다. 따라서 나머지 1번째, 5번째, 6번째 반찬은 제거해줘도 결과에 크게 영향을 미치지 않을 것이라 판단했다.  

In [317]:
# 중식에서는 1,5,6,7 번째 음식 제거, 석식에서는 1,5,6번째 음식 제거
def remove_specific_items(menu_list, indices):
    if isinstance(menu_list, list):
        return [item for i, item in enumerate(menu_list) if i not in indices]
    return menu_list


In [318]:
train['중식메뉴'] = train['중식메뉴'].apply(lambda x: remove_specific_items(x, [0, 4, 5, 6]))
train['석식메뉴'] = train['석식메뉴'].apply(lambda x: remove_specific_items(x, [0, 4, 5]))
test['중식메뉴'] = test['중식메뉴'].apply(lambda x: remove_specific_items(x, [0, 4, 5, 6]))
test['석식메뉴'] = test['석식메뉴'].apply(lambda x: remove_specific_items(x, [0, 4, 5]))

In [319]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,month,corona,shift_1day,day_gap
0,2016-02-01,5,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"[오징어찌개, 쇠불고기, 계란찜]","[육개장, 자반고등어구이, 두부조림]",1039.0,331.0,2,0,2016-02-02,0
1,2016-02-02,4,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"[김치찌개, 가자미튀김, 모둠소세지구이]","[어묵국, 유산슬, 아삭고추무침]",867.0,560.0,2,0,2016-02-03,0
2,2016-02-03,3,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"[팽이장국, 치킨핑거, 쫄면야채무침]","[청국장찌개, 황태양념구이, 고기전]",1017.0,573.0,2,0,2016-02-04,0
3,2016-02-04,2,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","[쇠고기무국, 주꾸미볶음, 부추전]","[우동, 멕시칸샐러드, 군고구마]",978.0,525.0,2,0,2016-02-05,0
4,2016-02-05,1,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"[떡국, 돈육씨앗강정, 우엉잡채]","[차돌박이찌개, 닭갈비, 감자소세지볶음]",925.0,330.0,2,0,2016-02-11,1


In [320]:
# 중식메뉴에서 각 항목을 분리하여 새로운 컬럼 생성
train['중식_국'] = train['중식메뉴'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
train['중식_메인1'] = train['중식메뉴'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
train['중식_메인2'] = train['중식메뉴'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else None)

test['중식_국'] = test['중식메뉴'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
test['중식_메인1'] = test['중식메뉴'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
test['중식_메인2'] = test['중식메뉴'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else None)

In [321]:
# 석식메뉴에서 각 항목을 분리하여 새로운 컬럼 생성
train['석식_국'] = train['석식메뉴'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
train['석식_메인1'] = train['석식메뉴'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
train['석식_메인2'] = train['석식메뉴'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else None)

test['석식_국'] = test['석식메뉴'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
test['석식_메인1'] = test['석식메뉴'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
test['석식_메인2'] = test['석식메뉴'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else None)

In [322]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', 'month', 'corona',
       'shift_1day', 'day_gap', '중식_국', '중식_메인1', '중식_메인2', '석식_국', '석식_메인1',
       '석식_메인2'],
      dtype='object')

In [323]:
test.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', 'month', 'corona', 'shift_1day',
       'day_gap', '중식_국', '중식_메인1', '중식_메인2', '석식_국', '석식_메인1', '석식_메인2'],
      dtype='object')

In [324]:
train = train.drop(['조식메뉴', '중식메뉴', '석식메뉴'], axis=1)
test = test.drop(['조식메뉴', '중식메뉴', '석식메뉴'], axis=1)

In [325]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,month,corona,shift_1day,day_gap,중식_국,중식_메인1,중식_메인2,석식_국,석식_메인1,석식_메인2
0,2016-02-01,5,2601,50,150,238,0.0,1039.0,331.0,2,0,2016-02-02,0,오징어찌개,쇠불고기,계란찜,육개장,자반고등어구이,두부조림
1,2016-02-02,4,2601,50,173,319,0.0,867.0,560.0,2,0,2016-02-03,0,김치찌개,가자미튀김,모둠소세지구이,어묵국,유산슬,아삭고추무침
2,2016-02-03,3,2601,56,180,111,0.0,1017.0,573.0,2,0,2016-02-04,0,팽이장국,치킨핑거,쫄면야채무침,청국장찌개,황태양념구이,고기전
3,2016-02-04,2,2601,104,220,355,0.0,978.0,525.0,2,0,2016-02-05,0,쇠고기무국,주꾸미볶음,부추전,우동,멕시칸샐러드,군고구마
4,2016-02-05,1,2601,278,181,34,0.0,925.0,330.0,2,0,2016-02-11,1,떡국,돈육씨앗강정,우엉잡채,차돌박이찌개,닭갈비,감자소세지볶음


In [326]:
train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']


test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']



In [327]:
train = train.drop(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', 'shift_1day'], axis=1)
test = test.drop(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', 'shift_1day'], axis=1)

In [328]:
train.head()

,일자,요일,중식계,석식계,month,corona,day_gap,중식_국,중식_메인1,중식_메인2,석식_국,석식_메인1,석식_메인2,출근,휴가비율,출장비율,야근비율,재택비율
0,2016-02-01,5,1039.0,331.0,2,0,0,오징어찌개,쇠불고기,계란찜,육개장,자반고등어구이,두부조림,2401.0,0.019223,0.057670,0.099125,0.0
1,2016-02-02,4,867.0,560.0,2,0,0,김치찌개,가자미튀김,모둠소세지구이,어묵국,유산슬,아삭고추무침,2378.0,0.019223,0.066513,0.134146,0.0
2,2016-02-03,3,1017.0,573.0,2,0,0,팽이장국,치킨핑거,쫄면야채무침,청국장찌개,황태양념구이,고기전,2365.0,0.021530,0.069204,0.046934,0.0
3,2016-02-04,2,978.0,525.0,2,0,0,쇠고기무국,주꾸미볶음,부추전,우동,멕시칸샐러드,군고구마,2277.0,0.039985,0.084583,0.155907,0.0
4,2016-02-05,1,925.0,330.0,2,0,1,떡국,돈육씨앗강정,우엉잡채,차돌박이찌개,닭갈비,감자소세지볶음,2142.0,0.106882,0.069589,0.015873,0.0


In [329]:
train.to_csv('D:/김동영/11_Github/mygit-1/데이터사이언스수업/과제/최종과제/train_preprocessed.csv', index=False)
test.to_csv('D:/김동영/11_Github/mygit-1/데이터사이언스수업/과제/최종과제/test_preprocessed.csv', index=False)